In [ ]:
import Utils
import pyDOE
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [2]:
''' Kriging for Robust Regularization'''
def kriging_minimax(train_data,test_data,Hyper):
    initial,lower,upper,optim = Hyper
    kernel =  RBF(initial, (lower, upper))
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=optim,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred

''' Kriging for Robust Regularization'''
def kriging_composite(train_data,test_data,Hyper):
    initial,lower,upper,optim = Hyper
    kernel =  RBF(initial, (lower, upper))
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=optim,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred

''' Latin HyperCube Sampling Design of Experiment for Hyper_Parameters Optimization'''
def DOE_Hyper():
    lhd = pyDOE.lhs(n=4, samples=1000, criterion='m')
    X1, X2 , X3, X4 = lhd[:,0], lhd[:,1] , lhd[:,2] , lhd[:,3] 
    X1 = Utils.linearscaletransform(X1,range_out=(5,1e2))
    X2 = Utils.linearscaletransform(X2,range_out=(1e-2 , 100))
    X3 = Utils.linearscaletransform(X3,range_out=(1e2 , 1e5))
    X4 = Utils.linearscaletransform(X4,range_out=(5 , 100))
    Hyper_Parameters = pd.DataFrame()
    Hyper_Parameters['initial'] = pd.Series(X1)
    Hyper_Parameters['lower_bound'] = pd.Series(X2)
    Hyper_Parameters['upper_bound'] = pd.Series(X3)
    Hyper_Parameters['n_restart_optim'] = pd.Series(X4)
    return Hyper_Parameters

## Kriging

In [4]:
X = [50,100,150,200,250,300,350,400,450,500]
Kriging_minimax = np.zeros([len(X),4])
Kriging_composite = np.zeros([len(X),4])
for k in range(len(X)):
    print (X[k])
    train = pd.read_csv(sys.path[1][:-7]+str('Training_Data_Sets\\'+str(X[k])+'Samples.csv')).iloc[:,1:]
    test = pd.read_csv(sys.path[1][:-7]+str('Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
    true_minimax = np.array(test.True_Minimax)
    true_composite = np.array(test.True_Composite)
    Hyper_Parameters = DOE_Hyper()
    mean_abs_error_minimax = np.zeros(Hyper_Parameters.shape[0])
    mean_abs_error_composite = np.zeros(Hyper_Parameters.shape[0])
    for i in range(Hyper_Parameters.shape[0]):
        temp = np.array(Hyper_Parameters.iloc[i,:])
        model_minimax,pred_k_m = kriging_minimax(train,test.iloc[:,:10],np.ceil(temp).astype(int))
        model_composite,pred_k_c = kriging_composite(train,test.iloc[:,:10],np.ceil(temp).astype(int))
        mean_abs_error_minimax[i] = np.mean((abs(true_minimax-pred_k_m) / abs(true_minimax) ) * 100)
        mean_abs_error_composite[i] = np.mean((abs( true_composite-pred_k_c) / abs(true_composite) ) * 100)
    Hyper_Parameters ['Mean_Error_Minimax'] = pd.Series(mean_abs_error_minimax)
    Hyper_Parameters ['Mean_Error_Composite'] = pd.Series(mean_abs_error_composite)
    Kriging_minimax [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-2],dtype=int)
    Kriging_composite [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),:-2],dtype=int)

50
100
150
200
250
300
350
400
450
500


In [5]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-1]

Best Configuration of Hyper Parameters for Robust Regularization


initial                  74.391425
lower_bound              99.582805
upper_bound           83935.253260
n_restart_optim          93.871892
Mean_Error_Minimax        1.798985
Name: 17, dtype: float64

In [6]:
print ('Best Configuration of Hyper Parameters for Robust Composition')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),[0,1,2,3,5]]

Best Configuration of Hyper Parameters for Robust Composition


initial                    10.207793
lower_bound                75.021221
upper_bound             28334.333835
n_restart_optim            45.032448
Mean_Error_Composite        0.719782
Name: 24, dtype: float64

In [7]:
Hyper_Parameters.to_csv('Result_Hyper_Params.csv')

In [8]:
pd.DataFrame(Kriging_composite).to_csv('composite.csv')
pd.DataFrame(Kriging_minimax).to_csv('Minimax.csv')

In [9]:
minimax = pd.read_csv('Minimax.csv').iloc[:,1:]
composite = pd.read_csv('composite.csv').iloc[:,1:]

In [10]:
np.array(composite)

array([[8.0000e+00, 0.0000e+00, 5.1597e+04, 6.4000e+01],
       [6.0000e+01, 8.3000e+01, 6.4168e+04, 7.3000e+01],
       [8.6000e+01, 9.9000e+01, 5.9640e+04, 6.8000e+01],
       [9.8000e+01, 7.4000e+01, 6.3818e+04, 9.6000e+01],
       [6.5000e+01, 9.9000e+01, 6.6514e+04, 3.8000e+01],
       [9.1000e+01, 8.3000e+01, 5.6558e+04, 3.5000e+01],
       [5.6000e+01, 6.5000e+01, 1.7680e+03, 8.0000e+00],
       [3.1000e+01, 6.0000e+01, 1.9103e+04, 6.0000e+00],
       [4.8000e+01, 9.9000e+01, 9.6540e+03, 4.4000e+01],
       [1.0000e+01, 7.5000e+01, 2.8334e+04, 4.5000e+01]])

In [11]:
np.array(minimax)

array([[8.0000e+00, 0.0000e+00, 5.1597e+04, 6.4000e+01],
       [7.1000e+01, 9.9000e+01, 5.1349e+04, 1.1000e+01],
       [8.6000e+01, 9.9000e+01, 5.9640e+04, 6.8000e+01],
       [7.5000e+01, 9.6000e+01, 8.1547e+04, 5.6000e+01],
       [6.5000e+01, 9.9000e+01, 6.6514e+04, 3.8000e+01],
       [3.8000e+01, 9.9000e+01, 9.0573e+04, 6.5000e+01],
       [8.0000e+01, 9.9000e+01, 9.3202e+04, 3.2000e+01],
       [8.1000e+01, 9.9000e+01, 9.4042e+04, 5.5000e+01],
       [4.8000e+01, 9.9000e+01, 9.6540e+03, 4.4000e+01],
       [7.4000e+01, 9.9000e+01, 8.3935e+04, 9.3000e+01]])